In [1]:
!pip install -U ortools

In [2]:
!pip install -U pyproj

In [3]:
# -*- coding: utf-8 -*-
import math
scale = input("何倍の精度にしますか")
scale = int(scale)
pole_radius = 6356752.314245                  # 極半径
equator_radius = 6378137.0

def cal_distance(latlon_kamata0,latlon_kamata1,latlon_yokosukachuo0,latlon_yokosukachuo1,scale):

    # 緯度経度をラジアンに変換
    lat_kamata = math.radians(latlon_kamata0)
    lon_kamata = math.radians(latlon_kamata1)
    lat_yokosukachuo = math.radians(latlon_yokosukachuo0)
    lon_yokosukachuo = math.radians(latlon_yokosukachuo1)

    lat_difference = lat_kamata - lat_yokosukachuo       # 緯度差
    lon_difference = lon_kamata - lon_yokosukachuo       # 経度差
    lat_average = (lat_kamata + lat_yokosukachuo) / 2    # 平均緯度

    e2 = (math.pow(equator_radius, 2) - math.pow(pole_radius, 2)) \
            / math.pow(equator_radius, 2)  # 第一離心率^2

    w = math.sqrt(1- e2 * math.pow(math.sin(lat_average), 2))

    m = equator_radius * (1 - e2) / math.pow(w, 3) # 子午線曲率半径

    n = equator_radius / w                         # 卯酉線曲半径

    distance = math.sqrt(math.pow(m * lat_difference, 2) \
                   + math.pow(n * lon_difference * math.cos(lat_average), 2)) # 距離計測

    print(distance / 1000)
    return scale * distance

if __name__ == '__main__':
    cal_distance(35.562479, 139.716073, 35.278699, 139.670040,scale)



何倍の精度にしますか10000
31.7612541505034


In [4]:
import pandas as pd

url="ST.csv"
df = pd.read_csv(url,encoding='shift-jis')

print(df.shape) # (12, 39)
print(df)
print(df['緯度'],df['経度'])
li=[[0]*len(df["緯度"]) for i in range(len(df["緯度"]))]
for from_node in range(len(df["緯度"])):
    for to_node in range(len(df['緯度'])):
        li[from_node][to_node]=cal_distance(df['緯度'][from_node], df['経度'][from_node], df['緯度'][to_node], df['経度'][to_node],scale)
print(li)

(47, 6)
    乗客数ランク「札幌の都市交通データ」（令和4年度(データは2021)）を加工して作成  路線名       駅名         緯度  \
0                                            2  南北線       大通  43.060444   
1                                            1  南北線     さっぽろ  43.066417   
2                                            1  東豊線     さっぽろ  43.066417   
3                                            3  東西線    新さっぽろ  43.038917   
4                                            4  南北線       麻生  43.109056   
5                                            5  東西線    西18丁目  43.057167   
6                                            6  東西線    西11丁目  43.058611   
7                                            7  東西線       琴似  43.075769   
8                                            8  東西線       白石  43.046208   
9                                            9  南北線     北24条  43.089944   
10                                          10  東西線      宮の沢  43.090028   
11                                          11  東西線     円山公園  43.055639   
12               

In [6]:
import plotly.graph_objs as go

trace = go.Scattermapbox(
    lon = df.経度,
    lat = df.緯度,
    mode = 'markers + text',
    text = df.駅名,
    marker = dict(size=df.乗客数/1000,
                  color="red",
                 ),
)

data = [trace]

fig = go.Figure(data)
fig.update_layout(mapbox_style="open-street-map",
                  mapbox_center_lat=43.0664,
                  mapbox_center_lon=141.3545,
                  mapbox={'zoom': 10},
                  margin={"r":0,"t":0,"l":0,"b":0},
)

fig.show()


In [7]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

de=df['乗客数']
li =  np.vectorize(int)(li)
print(de)
de[0]=0
def zusi(result_table):
        result_table['到着順'] = result_table.groupby('vehicle_id').cumcount()
        route_v0 = result_table.query('vehicle_id==0').merge(df, left_on='地点', right_index=True)
        route_v1 = result_table.query('vehicle_id==1').merge(df, left_on='地点', right_index=True)
        route_v2 = result_table.query('vehicle_id==2').merge(df, left_on='地点', right_index=True)
        route_v0 = pd.concat([route_v0,pd.DataFrame(route_v0.iloc[0:1])])
        route_v1 = pd.concat([route_v1,pd.DataFrame(route_v1.iloc[0:1])])
        route_v2 = pd.concat([route_v2,pd.DataFrame(route_v1.iloc[0:1])])
        depot = route_v0.iloc[0:1].copy()
        depot['累積距離'] = 0
        trace = []
        for route, color, name in zip([route_v0, route_v1,route_v2, depot],
                                    ['green', 'orange', 'blue', 'black'],
                                    ['vehicle1', 'vehicle2','vehicle3', 'depot']):
            trace_route = go.Scattermapbox(
                lon = route['経度'],
                lat = route['緯度'],
                mode = 'markers + text + lines',
                text = route['駅名'] + '<br>' +
                    '到着順: ' + route['到着順'].astype(str) + '<br>' +
                    'ここまでの累積載量: ' + route['累積積載量'].astype(str) + '<br>' +
                    '次地点到着までの累積距離: ' + route['累積距離'].astype(str) + 'm' ,
                marker = dict(size=route['乗客数']/1000),
                line = dict(color=color),
                name=name
            )
            trace.append(trace_route)

        data = trace
        fig = go.Figure(data)
        fig.update_layout(mapbox_style="open-street-map",
                        mapbox_center_lat=43.0664,
                        mapbox_center_lon=141.3545,
                        mapbox={'zoom': 10},
                        margin={"r":0,"t":0,"l":0,"b":0},
        )
        fig.show()
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] =li
    data["demands"] = de
    data["vehicle_capacities"] = [175239, 209647, 116974]
    data["num_vehicles"] = 3
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    result = dict()
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += (routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            ))/scale
            result[previous_index] = {'地点': node_index, '累積積載量': route_load, '累積距離': route_distance, 'vehicle_id': vehicle_id}
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load

    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")
    return result

def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        result=print_solution(data, manager, routing, solution)
        result_table = pd.DataFrame(result).T
        print(result_table)
        zusi(result_table)

if __name__ == "__main__":
    main()

0         0
1     38753
2     23694
3     16307
4     15853
5     13258
6     13227
7     12065
8     11813
9     11539
10    11426
11    11410
12    11116
13    10254
14     9985
15     9353
16     9153
17     9090
18     8895
19     8779
20     8458
21     8365
22     7823
23     7781
24     7749
25     7613
26     7124
27     7025
28     6892
29     6856
30     6696
31     6629
32     6520
33     6340
34     6277
35     5995
36     5771
37     5383
38     5139
39     4731
40     4511
41     4449
42     4235
43     4058
44     3999
45     3220
46     2501
Name: 乗客数, dtype: int64


/tmp/ipython-input-7-3819420545.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Objective: 656693436
Route for vehicle 0:
 0 Load(0) ->  2 Load(23694) ->  1 Load(62447) ->  44 Load(66446) ->  36 Load(72217) ->  9 Load(83756) ->  35 Load(89751) ->  4 Load(105604) ->  30 Load(112300) ->  34 Load(118577) ->  23 Load(126358) ->  26 Load(133482) ->  27 Load(140507) ->  45 Load(143727) ->  0 Load(143727)
Distance of the route: 15031.878999999997m
Load of the route: 143727

Route for vehicle 1:
 0 Load(0) ->  21 Load(8365) ->  15 Load(17718) ->  29 Load(24574) ->  42 Load(28809) ->  14 Load(38794) ->  8 Load(50607) ->  16 Load(59760) ->  31 Load(66389) ->  3 Load(82696) ->  40 Load(87207) ->  13 Load(97461) ->  18 Load(106356) ->  12 Load(117472) ->  24 Load(125221) ->  41 Load(129670) ->  17 Load(138760) ->  46 Load(141261) ->  22 Load(149084) ->  33 Load(155424) ->  28 Load(162316) ->  38 Load(167455) ->  43 Load(171513) ->  32 Load(178033) ->  39 Load(182764) ->  19 Load(191543) ->  0 Load(191543)
Distance of the route: 35372.68429999999m
Load of the route: 191543

Ro